In [1]:
import gym
import numpy as np
import time
from IPython import display

In [2]:
env = gym.make('FrozenLake-v0')

In [3]:
env.P[0][3] # Transition model

[(0.3333333333333333, 1, 0.0, False),
 (0.3333333333333333, 0, 0.0, False),
 (0.3333333333333333, 0, 0.0, False)]

In [4]:
env.observation_space.n

16

In [5]:
env.action_space.n

4

In [6]:
def play(env, policy, render=False):
    state = env.reset()
    total_reward = 0
    steps = 0
    done = False
    while not done:
        action = policy[state]
        next_state, reward, done, info = env.step(action)
        total_reward += reward
        steps += 1
        if render:
            env.render()
            time.sleep(0.2)
            if not done:
                display.clear_output(wait=True)
        state = next_state

    return (total_reward, steps)

In [7]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play(env, policy_0)

(0.0, 19)

In [8]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
play(env, policy_1, True)

  (Down)
SFFF
FHFH
FFFH
HFFG


(0.0, 11)

In [9]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
play(env, policy_2, True)

  (Down)
SFFF
FHFH
FFFH
HFFG


(0.0, 2)

In [10]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
play(env, policy_3, True)

  (Down)
SFFF
FHFH
FFFH
HFFG


(1.0, 55)

In [11]:
def play_multiple_times(env, policy, max_episodes):
    success = 0
    list_of_steps = []
    for i in range(max_episodes):
        total_reward, steps = play(env, policy)

        if total_reward > 0:
            success += 1
            list_of_steps.append(steps)

    print(f'Number of successes: {success}/{max_episodes}')
    print(f'Average number of steps: {np.mean(list_of_steps)}')

In [12]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play_multiple_times(env, policy_0, 1000)

Number of successes: 0/1000
Average number of steps: nan


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [13]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
play_multiple_times(env, policy_1, 1000)

Number of successes: 55/1000
Average number of steps: 11.272727272727273


In [14]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
play_multiple_times(env, policy_2, 1000)

Number of successes: 103/1000
Average number of steps: 14.320388349514563


In [15]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
play_multiple_times(env, policy_3, 1000)

Number of successes: 727/1000
Average number of steps: 37.95185694635488


# Policy - Interation

In [16]:
def policy_evaluation(env, policy, max_iters=500, gamma=0.9):
    # Initialize the values of all states to be 0
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # Update the value of each state
        for state in range(env.observation_space.n):
            action = policy[state]

            # Compute the q-value of the action
            q_value = 0
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * prev_v_values[next_state])

            v_values[state] = q_value # update v-value
        
        # Check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            print(f'Converged at {i}-th iteration.')
            break
    return v_values


In [17]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
v_values_0 = policy_evaluation(env, policy_0)
print(v_values_0)

Converged at 0-th iteration.
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [18]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
v_values_1 = policy_evaluation(env, policy_1)
print(v_values_1)

Converged at 48-th iteration.
[0.01904157 0.01519815 0.03161906 0.02371389 0.02538879 0.
 0.06648515 0.         0.05924054 0.13822794 0.18999823 0.
 0.         0.21152109 0.56684236 0.        ]


In [19]:
np.all(v_values_1 >= v_values_0)

True

In [20]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
v_values_2 = policy_evaluation(env, policy_2)
print(v_values_2)

Converged at 53-th iteration.
[0.02889625 0.01951972 0.03616977 0.0271268  0.04790519 0.
 0.07391985 0.         0.08288277 0.19339319 0.21022995 0.
 0.         0.35153135 0.62684674 0.        ]


In [21]:
np.all(v_values_2 >= v_values_1)

True

In [26]:
def policy_iteration(env,max_iters=500, gamma=0.9):
    # Initiallize 
      policy = np.ones(env.observation_space.n, dtype=np.int)
      for i in range(max_iters):
        # Compute the Value Function for the current policy
        v_values = policy_evaluation(env,policy,max_iters=500,gamma=0.9)
        policy_stable = True
        #loop each state
        for state in range(env.observation_space.n):
          old_action = policy[state]
          q_values = []
          # loop through each action
          for action in range(env.action_space.n):
              # loop each possible outcome
              q_value = 0
              for prob, next_state, reward, done in env.P[state][action]:
                   q_value += prob * (reward + gamma * v_values[next_state])
              q_values.append(q_value)
          # select the best action
          best_action = np.argmax(q_values)
          policy[state] = best_action

          if old_action!=best_action:
            policy_stable = False

        if policy_stable:
          return policy,v_values

# Values - Interation

In [27]:
def policy_extraction(env, v_values, gamma=0.9):
    # initialize
    policy = np.zeros(env.observation_space.n, dtype=np.int)

    # loop through each state in the environment
    for state in range(env.observation_space.n):
        q_values = []
        # loop through each action
        for action in range(env.action_space.n):
            q_value = 0
            # loop each possible outcome
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * v_values[next_state])
            
            q_values.append(q_value)
        
        # select the best action
        best_action = np.argmax(q_values)
        policy[state] = best_action
        
    return policy

In [28]:
def value_iteration(env, max_iters=500, gamma=0.9):
    # initialize
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # update the v-value for each state
        for state in range(env.observation_space.n):
            q_values = []
            
            # compute the q-value for each action that we can perform at the state
            for action in range(env.action_space.n):
                q_value = 0
                # loop through each possible outcome
                for prob, next_state, reward, done in env.P[state][action]:
                    q_value += prob * (reward + gamma * prev_v_values[next_state])
                
                q_values.append(q_value)
            
            # select the max q-values
            best_action = np.argmax(q_values)
            v_values[state] = q_values[best_action]
        
        # check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            print(f'Converged at {i}-th iteration.')
            break
    policy = policy_extraction(env,v_values,gamma)
    return policy,v_values

In [29]:
optimal_policy,optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)

Converged at 79-th iteration.


In [30]:
optimal_v_values

array([0.06888615, 0.06141054, 0.07440682, 0.05580409, 0.09185022,
       0.        , 0.11220663, 0.        , 0.14543286, 0.2474946 ,
       0.29961593, 0.        , 0.        , 0.3799342 , 0.63901926,
       0.        ])

In [31]:
optimal_policy

array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])

In [32]:
optimal_policy
play_multiple_times(env, optimal_policy, 1000)

Number of successes: 750/1000
Average number of steps: 36.010666666666665


# Compare policy- iteration and values-iteration

In [42]:
# Enviroment 'FrozenLake-v0'
environment = gym.make('FrozenLake-v0')
n_esp = 1000
# Policy Interation
print('Policy Interation')
start_time = time.time()
optimal_policy, optimal_v_values = policy_iteration(environment,max_iters=500, gamma=0.9)
end_time = time.time()
print('Time for run policy-iteration', end_time-start_time)
play_multiple_times(environment,optimal_policy,n_esp)

# Values Interation
print('Values Interation')
start_time = time.time()
optimal_policy, optimal_v_values = value_iteration(environment,max_iters=500, gamma=0.9)
end_time = time.time()
print('Time for run values-iteration', end_time-start_time)
play_multiple_times(environment,optimal_policy,n_esp)

Policy Interation
Converged at 32-th iteration.
Converged at 80-th iteration.
Time for run policy-iteration 0.019724369049072266
Number of successes: 705/1000
Average number of steps: 38.299290780141845
Values Interation
Converged at 79-th iteration.
Time for run values-iteration 0.05530142784118652
Number of successes: 731/1000
Average number of steps: 37.40766073871409


In [43]:
# Enviroment 'FrozenLake8x8-v0'
environment = gym.make('FrozenLake8x8-v0')
n_esp = 1000
# Policy Interation
print('Policy Interation')
start_time = time.time()
optimal_policy, optimal_v_values = policy_iteration(environment,max_iters=500, gamma=0.9)
end_time = time.time()
print('Time for run policy-iteration', end_time-start_time)
play_multiple_times(environment,optimal_policy,n_esp)

# Values Interation
print('Values Interation')
start_time = time.time()
optimal_policy, optimal_v_values = value_iteration(environment,max_iters=500, gamma=0.9)
end_time = time.time()
print('Time for run values-iteration', end_time-start_time)
play_multiple_times(environment,optimal_policy,n_esp)

Policy Interation
Converged at 46-th iteration.
Converged at 121-th iteration.
Converged at 117-th iteration.
Time for run policy-iteration 0.09007978439331055
Number of successes: 731/1000
Average number of steps: 71.71135430916553
Values Interation
Converged at 117-th iteration.
Time for run values-iteration 0.20308589935302734
Number of successes: 735/1000
Average number of steps: 71.80272108843538


In [44]:
# Enviroment 'Taxi-v3'
environment = gym.make('Taxi-v3')
n_esp = 1000
# Policy Interation
print('Policy Interation')
start_time = time.time()
optimal_policy, optimal_v_values = policy_iteration(environment,max_iters=500, gamma=0.9)
end_time = time.time()
print('Time for run policy-iteration', end_time-start_time)
play_multiple_times(environment,optimal_policy,n_esp)

# Values Interation
print('Values Interation')
start_time = time.time()
optimal_policy, optimal_v_values = value_iteration(environment,max_iters=500, gamma=0.9)
end_time = time.time()
print('Time for run values-iteration', end_time-start_time)
play_multiple_times(environment,optimal_policy,n_esp)

Policy Interation
Converged at 88-th iteration.
Converged at 97-th iteration.
Converged at 100-th iteration.
Converged at 101-th iteration.
Converged at 102-th iteration.
Converged at 103-th iteration.
Converged at 106-th iteration.
Converged at 109-th iteration.
Converged at 110-th iteration.
Converged at 111-th iteration.
Converged at 112-th iteration.
Converged at 115-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Time for run policy-iteration 2.8494961261749268
Number of successes: 1000/1000
Average number of steps: 13.157
Values Interation
Converged at 116-th iteration.
Time for run values-iteration 1.4531328678131104
Number of successes: 1000/1000
Average number of steps: 13.091


# Nhận xét

Thông qua kết quả trả về có thể thấy value-iteration trả về kết quả tốt hơn policy-iteration với số trường hợp thành công lớn hơn và số bước đi ít hơn policy - iteration. Ngược lại policy-iteration thường có thời gian đưa ra kết quả nhanh hơn value-iteration vì policy-iteration hội tụ trong ít lần lặp hơn value-iteration(value-iteration nặng hơn về mặt tính toán). Nhưng nhìn chung việc policy-iteration và value-iteration cái nào nhanh hơn còn phụ thuộc vào môi trường, với trường hợp env 'FrozenLake-v0' và 'FrozenLake8x8-v0' policy-iteration nhanh hơn values - iteration nhưng với env Taxi v3 policy-iteration cho ra kết quả chậm hơn và cả 2 đều đưa ra kết quả tốt